In [ ]:
# %tensorflow_version 2.x
# import tensorflow 
# print(tensorflow.__version__)
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
import numpy as np
#from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input,Embedding, Bidirectional, RepeatVector, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import model_to_dot, plot_model
from pickle import load
from numpy import array
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

In [3]:
# clean text
def read_text(filename):
    # open the file
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    file.close()
    return text
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents
data = read_text("deu.txt")
deu_eng = to_lines(data)
deu_eng = array(deu_eng)
n_sentences = 10000
deu_eng = deu_eng[:n_sentences,:2]
deu_eng[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in deu_eng[:,0]]
deu_eng[:,1] = [s.translate(str.maketrans('', '', string.punctuation)) for s in deu_eng[:,1]]
for i in range(len(deu_eng)):
    deu_eng[i,0] = deu_eng[i,0].lower()
    deu_eng[i,1] = deu_eng[i,1].lower()
eng_l = []
deu_l = []
for i in range(len(deu_eng)):
    eng_l.append(len(deu_eng[i,0].split())) 
    deu_l.append(len(deu_eng[i,1].split()))

eng_length = max(eng_l)
deu_length = max(deu_l)
print (eng_length)
print (deu_length)

5
9


In [4]:
def tokenization(lines):
      tokenizer = Tokenizer()
      tokenizer.fit_on_texts(lines)
      return tokenizer
eng_tokenizer = tokenization(deu_eng[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
print('English Vocabulary Size: %d' % eng_vocab_size)
deu_tokenizer = tokenization(deu_eng[:, 1])
deu_vocab_size = len(deu_tokenizer.word_index) + 1
print('Deutch Vocabulary Size: %d' % deu_vocab_size)

def encode_sequences(tokenizer, length, lines):
         # integer encode sequences
         seq = tokenizer.texts_to_sequences(lines)
         # pad sequences with 0 values
         seq = pad_sequences(seq, maxlen=length, padding='post')
         #seq = np.expand_dims(seq, axis=2)
         return seq

English Vocabulary Size: 2220
Deutch Vocabulary Size: 3603


In [5]:
from sklearn.model_selection import train_test_split

# split data into train and test set
train, test = train_test_split(deu_eng, test_size=0.1, random_state = 12)

trainX = encode_sequences(deu_tokenizer, deu_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = trainY.reshape((trainY.shape[0], trainY.shape[1], 1))

# prepare validation data
testX = encode_sequences(deu_tokenizer, deu_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = testY.reshape((testY.shape[0], testY.shape[1], 1))

print ("first training example")
print (train[0])
print ('input German encoding:\n', trainX[0])
print ('target English encoding:\n', trainY[0])

print ("training shapes")
print (trainX.shape)
print (trainY.shape)

print ("testing shapes")
print (testX.shape)
print (testY.shape)

first training example
['tom will hurry' 'tom wird sich beeilen']
input German encoding:
 [  2  37  28 452   0   0   0   0   0]
target English encoding:
 [[  1]
 [ 75]
 [179]
 [  0]
 [  0]]
training shapes
(9000, 9)
(9000, 5, 1)
testing shapes
(1000, 9)
(1000, 5, 1)


In [0]:
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    x = Input(shape = (src_timesteps,))
    encoder_inputs = Embedding(src_vocab+1, n_units, input_length=src_timesteps, mask_zero=True)(x)
    #encoder = LSTM(n_units)(encoder_inputs)  # first LSTM layer is a many-to-one
    encoder = LSTM(n_units)(encoder_inputs)
    encoder_output = RepeatVector(tar_timesteps)(encoder) # we need to repeat that one output across expected output 
    decoder_inputs = LSTM(n_units, return_sequences=True)(encoder_output) 
    decoder_outputs = TimeDistributed(Dense(tar_vocab, activation='softmax'))(decoder_inputs) # many-to-many type
    model = Model(input = [x], output = [decoder_outputs])
    return model

In [7]:
# define model
model = define_model(deu_vocab_size, eng_vocab_size, deu_length, eng_length, 256)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='my_model.png', show_shapes=True)
# fit model
filename = 'my-ed-model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=50, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=1)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  if __name__ == '__main__':


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 9, 256)            922624    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 2220)           570540    
Total params: 2,543,788
Trainable params: 2,543,788
Non-trainable params: 0
_________________________________________________

In [8]:
from nltk.translate.bleu_score import corpus_bleu

def get_word(n, tokenizer):
      for word, index in tokenizer.word_index.items():
          if index == n:
              return word
      return None

def prediction_translation(dataset, predictions, tokenizer):
    actual = []
    eng_predictions = []
    
    for i in range(len(predictions)):
        curr_prediction = []
        for j in range(len(predictions[i])):
            best_index = argmax(predictions[i][j])
            if best_index > 0:
                curr_prediction.append(get_word(best_index, tokenizer))
            else:
                break
        eng_predictions.append(curr_prediction)
    
    for t, s in dataset:
        actual.append([t.split()])
        
    return actual, eng_predictions

preds = model.predict(trainX)
actual, predicted = prediction_translation(train, preds, eng_tokenizer)

print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.951738
BLEU-2: 0.935278
BLEU-3: 0.857045
BLEU-4: 0.533102


In [9]:
preds = model.predict(testX)
actual, predicted = prediction_translation(test, preds, eng_tokenizer)

print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.566483
BLEU-2: 0.453349
BLEU-3: 0.371153
BLEU-4: 0.178195
